# 利用 Paddle Quantum 的 qchem 模块进行量子化学计算
_Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved._


qchem 是基于 Paddle Quantum 推出的用于量子化学研究的工具集。qchem 为量子化学领域的研究者提供了一系列工具，使他们可以利用量子计算方法完成量子化学任务。与此同时，qchem 也提供了方便开发者进行功能拓展的方式。目前，qchem 正处于开发之中，您可以将需求和建议通过 Github 的 issue 或 pull request 反馈给我们，我们会及时给出回复。

## 分子基态能量计算
qchem 为量子化学计算提供了很多便捷的工具。目前，qchem 模块支持下列分子波函数模版线路：
* Hardware Efficient ansatz[<sup>1</sup>](#refer-1),
* Slater determinant ansatz[<sup>2</sup>](#refer-2),
* Unitary Coupled Cluster singles and doubles (UCCSD) ansatz[<sup>3</sup>](#refer-3).

让我们从具体的例子出发了解 qchem 的使用方法，下面我们演示了利用 qchem 求解氢分子基态的过程。

In [2]:
import paddle_quantum as pq
from paddle_quantum import qchem as pq_qchem
import warnings
warnings.filterwarnings("ignore")

接下来，我们需要提供量子化学计算需要用到的一些分子性质，包括：分子的几何结构、分子的电荷、计算需要用到的量子化学基函数等。

In [3]:
# 定义氢分子的几何结构，长度单位为埃
h2_geometry = "H 0.0 0.0 0.0; H 0.0 0.0 0.74"
basis_set = "sto-3g"
multiplicity = 1
charge = 0

然后，我们需要为氢分子选择一种波函数模版线路。我们选择 `UCCSDModel` 作为模版线路并且用 `MolEnergyLoss` 作为损失函数。`UCCSDModel` 需要用 Trotter-Suzuki 方法构造其量子线路，关于 Trotter-Suzuki 方法，感兴趣的读者可以阅读这篇[教程](https://qml.baidu.com/tutorials/quantum-simulation/hamiltonian-simulation-with-product-formula.html)。

In [ ]:
# 构建 UCCSD 线路.
n_qubits = 4
n_electrons = 2
uccsd_ansatz = pq_qchem.UCCSDModel(n_qubits, n_electrons, n_trotter_steps=3)

# 设置损失函数
loss_fn = pq_qchem.MolEnergyLoss(h2_geometry, basis_set)

完成上面的步骤之后，我们可以按照[教程](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html)中的方法利用 paddlepaddle 中的优化器来训练参数化量子线路。

In [5]:
# 选择 paddlepaddle 中的 Adam 优化器
import paddle

optimizer = paddle.optimizer.Adam(parameters=uccsd_ansatz.parameters(), learning_rate=0.1)

# 制备初始量子态, e.g. |0000>
init_state = pq.state.computational_basis(n_qubits, 0)

# 定义优化步数
num_itr = 100
for itr in range(0, num_itr):
    # 运行量子线路得到末态
    state = uccsd_ansatz(init_state)
    # 计算损失函数，即期望值
    loss = loss_fn(state)
    # 反向传播梯度
    loss.backward()
    # 通过loss值更新参数
    optimizer.minimize(loss)
    # 清除当前梯度
    optimizer.clear_grad()
    print(f"The iter is {itr:3d}, loss is {loss.item():3.5f}.")
print("The theoretical value is -1.137283834485513.")

The iter is   0, loss is 0.71510.
The iter is   1, loss is 0.63558.
The iter is   2, loss is 0.13140.
The iter is   3, loss is -0.20547.
The iter is   4, loss is -0.59966.
The iter is   5, loss is -0.30310.
The iter is   6, loss is -0.69676.
The iter is   7, loss is -0.75225.
The iter is   8, loss is -0.75798.
The iter is   9, loss is -0.92072.
The iter is  10, loss is -0.89658.
The iter is  11, loss is -0.90863.
The iter is  12, loss is -0.96792.
The iter is  13, loss is -0.95464.
The iter is  14, loss is -0.96280.
The iter is  15, loss is -1.00236.
The iter is  16, loss is -1.02617.
The iter is  17, loss is -1.06962.
The iter is  18, loss is -1.06430.
The iter is  19, loss is -1.04589.
The iter is  20, loss is -1.06411.
The iter is  21, loss is -1.05254.
The iter is  22, loss is -1.04040.
The iter is  23, loss is -1.07904.
The iter is  24, loss is -1.10329.
The iter is  25, loss is -1.09534.
The iter is  26, loss is -1.10908.
The iter is  27, loss is -1.10671.
The iter is  28, loss i

现在你可以把波函数模版替换为 `HardwareEfficientModel`，然后尝试比较一下使用两种方法计算出来的基态能量。

## 利用Hatree Fock方法计算分子基态能量
Hartree Fock方法是量子化学中非常重要的方法。如果要利用qchem模块运行Hartree Fock方法的话，我们只需要把前面的波函数模版线路换成`RHFSlaterDeterminantModel`，并把损失函数换成`RHFEnergyLoss`（注意：使用Hartree Fock方法需要安装PySCF，`pip install -U pyscf`）。

In [6]:
# 构建Hartree Fock线路
n_qubits = 4
n_electrons = 2
hartreefock_ansatz = pq_qchem.RHFSlaterDeterminantModel(n_qubits, n_electrons)

# 设置Hartree Fock损失函数
loss_fn = pq_qchem.RHFEnergyLoss(h2_geometry, basis_set)

Overwritten attributes  multiplicity  of <class 'pyscf.gto.mole.Mole'>


In [7]:
# 选择 paddlepaddle 中的 Adam 优化器
import paddle

optimizer = paddle.optimizer.Adam(parameters=hartreefock_ansatz.parameters(), learning_rate=0.1)

# 制备初始量子态, e.g. |1100>
init_state = pq.state.computational_basis(n_qubits, 12)

# 定义优化步数
num_itr = 100
for itr in range(0, num_itr):
    # 运行量子线路得到末态
    state = hartreefock_ansatz(init_state)
    # 计算损失函数，即期望值
    loss = loss_fn(state)
    # 反向传播梯度
    loss.backward()
    # 通过loss值更新参数
    optimizer.minimize(loss)
    # 清除当前梯度
    optimizer.clear_grad()
    print(f"The iter is {itr:3d}, loss is {loss.item():3.5f}.")
print("The theoretical value is -1.11675.")

The iter is   0, loss is 0.37967.
The iter is   1, loss is 0.31168.
The iter is   2, loss is 0.22180.
The iter is   3, loss is 0.10975.
The iter is   4, loss is -0.02357.
The iter is   5, loss is -0.17552.
The iter is   6, loss is -0.34124.
The iter is   7, loss is -0.51346.
The iter is   8, loss is -0.68254.
The iter is   9, loss is -0.83712.
The iter is  10, loss is -0.96538.
The iter is  11, loss is -1.05717.
The iter is  12, loss is -1.10677.
The iter is  13, loss is -1.11545.
The iter is  14, loss is -1.09212.
The iter is  15, loss is -1.05087.
The iter is  16, loss is -1.00622.
The iter is  17, loss is -0.96914.
The iter is  18, loss is -0.94574.
The iter is  19, loss is -0.93791.
The iter is  20, loss is -0.94471.
The iter is  21, loss is -0.96343.
The iter is  22, loss is -0.99040.
The iter is  23, loss is -1.02149.
The iter is  24, loss is -1.05253.
The iter is  25, loss is -1.07977.
The iter is  26, loss is -1.10035.
The iter is  27, loss is -1.11271.
The iter is  28, loss is

---
## 参考文献

[1] Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." [Nature 549.7671 (2017): 242-246.](https://www.nature.com/articles/nature23879)

[2] Arute, Frank, et al. "Hartree-Fock on a superconducting qubit quantum computer." [Science 369.6507 (2020): 1084-1089.](https://www.science.org/doi/10.1126/science.abb9811)